## MS-SMC' validation

In [4]:
from concurrent.futures import ProcessPoolExecutor
from typing import Tuple
import numpy as np
import pandas as pd
import ipcoal
import toyplot, toyplot.svg, toyplot.png
import toytree
from scipy import stats

### Parameters

In [11]:
RECOMB = 2e-9
SPECIES_TREE_HEIGHT = 1e6
NEFF_MIN = 50_000
NEFF_MAX = 500_000
NEFF_VALS = 10
SEED = 123
NLOCI = 4

### Generate a species tree for model 1 tip, 2 tips, or 8 tips

In [12]:
sptree = toytree.tree("(a);")
sptree = sptree.mod.edges_scale_to_root_height(treeheight=SPECIES_TREE_HEIGHT, include_stem=True)
sptree.draw('p');

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="255.0px" viewBox="0 0 350.0 255.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t7821e1b26943499e975f1d7db8431f7d"> 0 1 a 0 250000 500000

In [13]:
sptree = toytree.rtree.baltree(2)
sptree = sptree.mod.edges_scale_to_root_height(treeheight=SPECIES_TREE_HEIGHT, include_stem=True)
sptree.draw('p');

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="260.0px" viewBox="0 0 350.0 260.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t70bc5cf5fb794a3ebf32c59a128d461f"> 0 1 2 r0 r1 0 250000 500000

In [14]:
sptree = toytree.rtree.baltree(8)
sptree = sptree.mod.edges_scale_to_root_height(treeheight=SPECIES_TREE_HEIGHT, include_stem=True)
sptree.draw('p');

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="260.0px" viewBox="0 0 350.0 260.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t55f27c7e697b473097e43b9e8c0257f6"> 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 r0 r1 r2 r3 r4 r5 r6 r7 0 250000 500000 750000

In [25]:
def get_sptree(ntips: int) -> toytree.ToyTree:
    """Return a balanced species tree with the specified number of tips and even branch lengths."""
    if ntips == 1:
        sptree = toytree.tree("(a);")
    else:
        sptree = toytree.rtree.baltree(ntips)
    sptree = sptree.mod.edges_scale_to_root_height(SPECIES_TREE_HEIGHT, include_stem=True)
    return sptree

### Get waiting distances to event x:


In [26]:
def get_sim_and_analytical(sptree: toytree.ToyTree, nsamples: int, neff: int, nsites: int, seed: int, smc: bool) -> Tuple:
    """Return the prob and waiting distance for tree-change and topo-change for analytical and 
    simulations.
    
    Analytical results are computed for the first genealogy in a locus. Simulated results 
    are found by examining subsequent genealogies in the tree sequence until each event
    type is observed. 
    
    This uses some global variables (see above).
    
    Parameters
    ----------
    sptree: ToyTree
        A species tree with edge lengths in units of generations
    nsamples: int
        Number of haploid sampler per species tree tip.
    neff: int
        A diploid effective population size applied to all sptree edges.
    nsites: int
        Number of linked sites to simulate.
    seed: int
        A seed for the random number generator
    """
    model = ipcoal.Model(
        sptree, 
        Ne=neff, 
        seed_trees=seed, 
        nsamples=nsamples, 
        recomb=RECOMB,
        record_full_arg=True,
        ancestry_model="smc_prime" if smc else "hudson",  # use default (Hudson)
    )
    imap = model.get_imap_dict()
    
    # store results in arrays
    # 0 = smc_tree_probs
    # 1 = smc_topo_probs
    # 2 = smc_tree_dists
    # 3 = smc_topo_dists
    # 4 = sim_tree_dists
    # 5 = sim_topo_dists
    # 6 = sum_edge_lengths of genealogy 0
    # 7 = event type
    results = np.zeros(shape=(NLOCI, 8))
    
    # iterate over number of loci (replicates)
    for lidx in range(NLOCI):
        
        # get FULL tree sequence given the parameterized ipcoal.Model
        ts = next(model._get_tree_sequence_generator(nsites=nsites))
        
        # get SIMPLE tree sequence for fetching simplified trees at breakpoints
        # from the FULL `ts`, since no-change breakpoints disappear in `sts`.
        sts = ts.simplify(filter_sites=False)

        # get the starting tree and its sum branch lengths
        tree0 = sts.at(0, sample_lists=True)
        tsumlen0 = tree0.get_total_branch_length()
        results[lidx, 6] = tsumlen0

        # compute analytical probabilities of change given tree0
        toy0 = toytree.tree(tree0.as_newick(node_labels=model.tipdict))
        results[lidx, 0] = ipcoal.smc.get_probability_of_tree_change(model.tree, toy0, imap)
        results[lidx, 1] = ipcoal.smc.get_probability_of_topology_change(model.tree, toy0, imap)
        
        # compute analytical E[waiting distances] given tree0
        tree_rate = tsumlen0 * results[lidx, 0] * RECOMB
        topo_rate = tsumlen0 * results[lidx, 1] * RECOMB
        results[lidx, 2] = stats.expon.mean(scale=1 / tree_rate)
        results[lidx, 3] = stats.expon.mean(scale=1 / topo_rate)
        
        # get the next simulated tree (simplified tree at end of interval 0 of non-simple ts)
        tree1 = sts.at(ts.at(0).interval[1], sample_lists=True)
        
        # store type of first event: 0=no-change, 1=tree-change, 2=topo-change
        if tree0.kc_distance(tree1, lambda_=1):      # diff in blens only
            if tree0.kc_distance(tree1, lambda_=0):  # diff in topology only
                results[lidx, 7] = 2
                results[lidx, 4] = tree1.interval[0]
                results[lidx, 5] = tree1.interval[0]
            else:
                results[lidx, 7] = 1
                results[lidx, 4] = tree1.interval[0]
        else:
            results[lidx, 7] = 0

        # if waiting distances already found then continue to next locus
        if results[lidx, 5]:
            continue
        
        # iterate over trees from index 2 -> End to get waiting distances until the 
        # first observed tree and topo-change events from simulation.
        for idx in range(2, 999999):
            interval = ts.at_index(idx).interval
            tree1 = sts.at(interval[0], sample_lists=True)
                
            # if no difference in branch lengths then go to next idx
            if tree0.kc_distance(tree1, lambda_=1):     # diff in blens only
                if not results[lidx, 4]:
                    results[lidx, 4] = tree1.interval[0]
                if tree0.kc_distance(tree1, lambda_=0):  # diff in topology only
                    results[lidx, 5] = tree1.interval[0]
                    break
    return results

In [29]:
# example results array
sptree = get_sptree(8)
get_sim_and_analytical(sptree, nsamples=1, neff=100000, nsites=500_000, seed=123, smc=False)

array([[5.54764539e-01, 1.53804181e-01, 1.57655692e+02, 5.68656759e+02,
        1.03000000e+02, 2.37900000e+03, 5.71678234e+06, 1.00000000e+00],
       [5.52203657e-01, 2.80267923e-01, 1.86793057e+02, 3.68032873e+02,
        9.90000000e+01, 9.90000000e+01, 4.84741270e+06, 0.00000000e+00],
       [5.38887852e-01, 2.11229182e-01, 2.04351239e+02, 5.21340846e+02,
        1.11400000e+03, 1.11400000e+03, 4.54040231e+06, 0.00000000e+00],
       [5.46230438e-01, 1.19758179e-01, 2.23907949e+02, 1.02126917e+03,
        4.18000000e+02, 8.74000000e+02, 4.08812904e+06, 0.00000000e+00]])

## ----

## Compare results of the Hudson and SMC' methods

In [39]:
def distribute_jobs(sptree: toytree.ToyTree, nsamples: int):
    """Parallelize get_sim_and_analytical() function across Ne values and reps.    
    """
    # compare smc and full
    smc_results = np.zeros((NLOCI * NREPS, NEFF_VALS, 8))
    full_results = np.zeros((NLOCI * NREPS, NEFF_VALS, 8))

    # Ne values to test over
    nes = np.linspace(NEFF_MIN, NEFF_MAX, NEFF_VALS).astype(int)

    # run jobs in parallel to fill array
    rasyncs = {}
    with ProcessPoolExecutor(max_workers=50) as pool:

        # apply a different seed to each rep
        rng = np.random.default_rng(SEED)
        for rep in range(NREPS):
            seed = rng.integers(1e9)

            # apply same seed for each diff value of Ne
            for nidx, neff in enumerate(nes):

                # submit smc' jobs
                kwargs = {
                    "sptree": sptree,
                    "nsamples": nsamples,
                    "neff": neff, 
                    "nsites": int(550000 - neff), 
                    "seed": seed, 
                    "smc": True,
                }
                rasyncs[(nidx, 'smc', rep)] = pool.submit(get_sim_and_analytical, **kwargs)
                
                # submit full (non-smc') jobs
                kwargs = {
                    "sptree": sptree, 
                    "nsamples": nsamples,
                    "neff": neff, 
                    "nsites": int(550000 - neff), 
                    "seed": seed, 
                    "smc": False,
                }
                rasyncs[(nidx, 'full', rep)] = pool.submit(get_sim_and_analytical, **kwargs)

    # collect results into large res array
    for key, future in rasyncs.items():
        nidx, name, rep = key
        ival = slice(NLOCI * rep, NLOCI * (rep + 1))
        if name == 'smc':
            iresults = future.result()
            smc_results[ival, nidx, :] = iresults
        else:
            iresults = future.result()
            full_results[ival, nidx, :] = iresults
           
    np.save(f"./validation_full_{sptree.ntips}pops.npy", full_results)
    np.save(f"./validation_smc_{sptree.ntips}pops.npy", smc_results)

### RUN it

In [40]:
# 10K reps for each job, split into 100 parallelizable jobs
NLOCI = 10
NREPS = 10

In [41]:
sptree = get_sptree(1)
distribute_jobs(sptree, nsamples=8)

In [110]:
sptree = get_sptree(2)
distribute_jobs(sptree, nsamples=4)

In [106]:
sptree = get_sptree(8)
distribute_jobs(sptree, nsamples=1)

### Load results

In [42]:
def load_data(path):
    """Return large array results summarized into a dataframe.
    
    # 0 = smc_tree_probs
    # 1 = smc_topo_probs
    # 2 = smc_tree_dists
    # 3 = smc_topo_dists
    # 4 = sim_tree_dists
    # 5 = sim_topo_dists
    # 6 = sum_edge_lengths of genealogy 0
    # 7 = event type
    """
    results = np.load(path)  
    
    newdata = pd.DataFrame(
        data={
            'tree_smc_prob': results[:, :, 0].mean(axis=0),
            'topo_smc_prob': results[:, :, 1].mean(axis=0),
            'tree_smc_dist': results[:, :, 2].mean(axis=0),
            'topo_smc_dist': results[:, :, 3].mean(axis=0),       
            'tree_sim_prob': (results[:, :, 7] != 0).mean(0),
            'topo_sim_prob': (results[:, :, 7] == 2).mean(0),
            'tree_sim_dist': results[:, :, 4].mean(axis=0),
            'topo_sim_dist': results[:, :, 5].mean(axis=0),

            'tree_smc_prob_CI95':  tuple(zip(
                np.percentile(results[:, :, 0], 2.5, axis=0).round(5),
                np.percentile(results[:, :, 0], 97.5, axis=0).round(5),
            )),
            'topo_smc_prob_CI95':  tuple(zip(
                np.percentile(results[:, :, 1], 2.5, axis=0).round(5),
                np.percentile(results[:, :, 1], 97.5, axis=0).round(5),
            )),

            'tree_smc_dist_CI95':  tuple(zip(
                np.percentile(results[:, :, 2], 2.5, axis=0).round(5),
                np.percentile(results[:, :, 2], 97.5, axis=0).round(5),
            )),
            'topo_smc_dist_CI95':  tuple(zip(
                np.percentile(results[:, :, 3], 2.5, axis=0).round(5),
                np.percentile(results[:, :, 3], 97.5, axis=0).round(5),
            )),
            'tmrca': results[:, :, 6].mean(axis=0),
            
            # error 
            'tree_dist_error': abs(results[:, :, 4] - results[:, :, 2]).mean(axis=0),
            'tree_dist_error_perc': (
                abs(results[:, :, 4] - results[:, :, 2]) / 
                results[:, :, 4]
            ).mean(axis=0),
            'topo_dist_error': abs(results[:, :, 5] - results[:, :, 3]).mean(axis=0),
            'topo_dist_error_perc': (
                abs(results[:, :, 5] - results[:, :, 3]) /
                results[:, :, 5]
            ).mean(axis=0),
            'tree_dist_error_perc_sem': stats.sem(abs(results[:, :, 4] - results[:, :, 2]) /
                results[:, :, 4], axis=0),
            'topo_dist_error_perc_sem': stats.sem(abs(results[:, :, 5] - results[:, :, 3]) /
                results[:, :, 5], axis=0),
        },
        index=np.linspace(NEFF_MIN, NEFF_MAX, NEFF_VALS).astype(int),
    )
    newdata['nsamples'] = NLOCI * NREPS
    return newdata

In [334]:
data = load_data("./validation_full_2pops.npy")
data

,tree_smc_prob,topo_smc_prob,tree_smc_dist,topo_smc_dist,tree_sim_prob,topo_sim_prob,tree_sim_dist,topo_sim_dist,tree_smc_prob_CI95,topo_smc_prob_CI95,tree_smc_dist_CI95,topo_smc_dist_CI95,tmrca,tree_dist_error,tree_dist_error_perc,topo_dist_error,topo_dist_error_perc,tree_dist_error_perc_sem,topo_dist_error_perc_sem,nsamples
50000,0.478757,0.111213,676.268799,3654.401925,0.4774,0.1093,675.3757,3582.1332,"(0.35682, 0.58968)","(0.03462, 0.23607)","(400.46634, 1094.71949)","(1130.48907, 9634.67349)",1.631340e+06,505.554692,5.853635,2699.725566,7.403688,0.302180,0.615305,10000
100000,0.646151,0.225855,363.881016,1321.674949,0.6474,0.2282,362.5759,1303.2978,"(0.53602, 0.72513)","(0.07393, 0.45016)","(219.17838, 579.45744)","(411.92082, 3439.35414)",2.258646e+06,268.997142,5.543616,968.946987,7.391848,0.255541,0.543980,10000
150000,0.709904,0.313065,260.326753,701.879038,0.7104,0.3126,265.0879,717.6677,"(0.6106, 0.77856)","(0.11164, 0.51751)","(154.84618, 415.39722)","(285.81673, 1775.83932)",2.897600e+06,193.774727,4.547354,528.130149,5.009915,0.179925,0.244005,10000
200000,0.740947,0.367595,207.779292,471.085088,0.7430,0.3687,210.0198,475.3567,"(0.64008, 0.80942)","(0.14678, 0.56616)","(119.75943, 332.09229)","(216.09206, 1117.71679)",3.512331e+06,155.109291,4.576590,353.180564,5.537589,0.177342,0.277451,10000
250000,0.758430,0.399244,175.429929,365.796188,0.7595,0.4050,175.4391,364.2903,"(0.64994, 0.82855)","(0.16764, 0.59391)","(101.35483, 291.65201)","(179.65041, 839.06861)",4.090813e+06,128.001204,4.574884,268.482098,5.378818,0.169407,0.329674,10000
300000,0.770349,0.421542,152.285179,300.711882,0.7724,0.4228,154.2638,304.1593,"(0.65554, 0.84176)","(0.18132, 0.61643)","(86.30296, 252.68551)","(152.85254, 656.09956)",4.672879e+06,112.047357,4.324776,223.176940,5.317500,0.153298,0.256465,10000
350000,0.778406,0.434678,134.704801,257.274349,0.7810,0.4326,135.8442,259.2106,"(0.66137, 0.85099)","(0.19871, 0.63489)","(75.19905, 227.33365)","(133.13225, 514.27454)",5.245334e+06,98.784750,4.257851,191.532145,4.968878,0.146660,0.214669,10000
400000,0.783426,0.442923,120.886196,227.550116,0.7860,0.4406,121.9483,228.4511,"(0.65704, 0.85879)","(0.19749, 0.64864)","(66.14219, 206.37715)","(118.10011, 444.08066)",5.849824e+06,88.909483,4.069404,167.146392,4.562119,0.128534,0.174756,10000
450000,0.789172,0.450712,110.766089,204.429160,0.7896,0.4551,112.1042,209.1004,"(0.66305, 0.86532)","(0.20441, 0.65532)","(60.65237, 189.04474)","(109.50605, 387.53868)",6.347644e+06,81.431988,4.071231,154.139916,4.597892,0.124830,0.170724,10000
500000,0.791365,0.452936,101.172475,186.493868,0.7933,0.4496,101.1601,185.2469,"(0.6602, 0.86852)","(0.20559, 0.66422)","(54.35087, 175.71241)","(99.2669, 354.14556)",6.969181e+06,73.586737,3.775336,135.633137,4.189815,0.114481,0.151901,10000


# Plot results

### Plot probabilities

In [44]:
def plot_probs(data, topo: bool=False, smc: bool=False):
    """Plot probabilities of tree or topology change.
    
    """      
    smckey = "topo_smc_" if topo else "tree_smc_"
    simkey = "topo_sim_" if topo else "tree_sim_"
    color = toytree.color.COLORS2[0]
    canvas = toyplot.Canvas(width=350, height=300)
    
    # setup axes
    axb = canvas.cartesian(margin=65)
    axt = axb.share("y")
    axb.x.label.text = "N<sub>e</sub> (diploid)"
    axt.x.label.text = "Sptree edge lengths (coal units)"
    axb.y.label.text = "P(topo-change | S,G)" if topo else "P(tree-change | S,G)"
    axb.y.domain.max = 1
    axb.y.domain.min = 0
    axb.x.domain.min = 0
    
    # style axes
    for ax in (axb.x, axt.x, axb.y):
        ax.domain.show = False
        ax.ticks.show = True
        ax.ticks.near = 7.5
        ax.ticks.far = 0
        ax.ticks.labels.offset = 12
        ax.ticks.labels.style["font-size"] = 16
        ax.label.offset = 35
        ax.label.style["font-size"] = 16
        ax.ticks.style["stroke-width"] = 3
        ax.spine.style["stroke-width"] = 3
        
    avgdist = np.min([i.dist for i in sptree if not i.is_root()])
    axb.x.ticks.locator = toyplot.locator.Explicit(
        np.linspace(50_000, 500_000, 4),
        #np.linspace(0.5, 5, 5),
    )
    axt.x.ticks.locator = toyplot.locator.Explicit(
        np.linspace(50_000, 500_000, 4),
        (avgdist / (2 * np.linspace(50_000, 500_000, 4))).round(1),
    )
        
    # plot data
    axb.hlines([0, 0.5, 1.0], style={"stroke": "grey", "stroke-width": 2, "stroke-dasharray": "6,6", "stroke-opacity": 0.7})
    axb.fill(
       data.index,
       [i[0] for i in data[smckey + "prob_CI95"]],
       [i[1] for i in data[smckey + "prob_CI95"]],
       opacity=0.33,
    )
    axb.plot(data.index, data[smckey + "prob"], stroke_width=2, color=color)
    style = dict(opacity=0.8, color='black', mstyle={"stroke": "none"})
    marks = [
        axb.scatterplot(data.index, data[smckey + "prob"], size=12, color=color, opacity=0.7),
        axb.scatterplot(data.index, data[simkey + "prob"], size=6, marker='s', **style),
    ]
    return canvas

In [45]:
for nspecies in [1, 2, 8]:
    for smc in [True, False]:
        for topo in [True, False]:
            data = load_data(f"./validation_full_{nspecies}pops.npy")
            canvas = plot_probs(data, topo=topo, smc=smc)
            ttt = 'topo' if topo else 'tree'
            #toyplot.svg.render(canvas, f"../manuscript/figures/alternatives/validation-probs-{nspecies}pops-{ttt}.svg")


<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t7345f1a8044c42ee9f1e4ed7f66ef539"> 50000 200000 350000 500000 N e (diploid) 0.0 0.5 1.0 P(topo-change | S,G) 5.0 1.2 0.7 0.5 Sptree edge lengths (coal units)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t64729ea79ac041f8a60ad10aebba1663"> 50000 200000 350000 500000 N e (diploid) 0.0 0.5 1.0 P(tree-change | S,G) 5.0 1.2 0.7 0.5 Sptree edge lengths (coal units)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t6adf1b801a394f08b5e656d956e98771"> 50000 200000 350000 500000 N e (diploid) 0.0 0.5 1.0 P(topo-change | S,G) 5.0 1.2 0.7 0.5 Sptree edge lengths (coal units)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t29130b48b4cd4c3987c914176098fed7"> 50000 200000 350000 500000 N e (diploid) 0.0 0.5 1.0 P(tree-change | S,G) 5.0 1.2 0.7 0.5 Sptree edge lengths (coal units)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t348673786e3a4a07910843d5ecbb7371"> 50000 200000 350000 500000 N e (diploid) 0.0 0.5 1.0 P(topo-change | S,G) 5.0 1.2 0.7 0.5 Sptree edge lengths (coal units)

...

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t38cf17e1fcd349f9b114724899c434a7"> 50000 200000 350000 500000 N e (diploid) 0.0 0.5 1.0 P(tree-change | S,G) 5.0 1.2 0.7 0.5 Sptree edge lengths (coal units)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t4f79c18375bd4abb8622c78e32583283"> 50000 200000 350000 500000 N e (diploid) 0.0 0.5 1.0 P(topo-change | S,G) 5.0 1.2 0.7 0.5 Sptree edge lengths (coal units)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tb3c80ef6f7fe49a98b234e4920178278"> 50000 200000 350000 500000 N e (diploid) 0.0 0.5 1.0 P(tree-change | S,G) 5.0 1.2 0.7 0.5 Sptree edge lengths (coal units)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t7a6ed00dc20442a581cfdc3d2d6b3d0b"> 50000 200000 350000 500000 N e (diploid) 0.0 0.5 1.0 P(topo-change | S,G) 5.0 1.2 0.7 0.5 Sptree edge lengths (coal units)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t0bf559912c8946f5a78603972165ff30"> 50000 200000 350000 500000 N e (diploid) 0.0 0.5 1.0 P(tree-change | S,G) 5.0 1.2 0.7 0.5 Sptree edge lengths (coal units)

## Plot distances

In [47]:
def plot_distances(data, topo: bool=False, log: bool=True, smc: bool=False):
    """Plot waiting distances until tree or topology change.
    
    """
    smckey = "topo_smc_" if topo else "tree_smc_"
    simkey = "topo_sim_" if topo else "tree_sim_"
    color = toytree.color.COLORS2[0]
    canvas = toyplot.Canvas(width=350, height=300)
    
    # setup axes
    axb = canvas.cartesian(margin=65)
    axt = axb.share("y")#, yscale="log")
    axb.x.label.text = "N<sub>e</sub> (diploid)"
    axt.x.label.text = "Sptree edge lengths (coal units)"
    axb.y.label.text = "Distance to topo-change" if topo else "Distance to tree-change"
    axb.x.domain.min = 0
        
    if log:
        axt.y.scale = "log"
        axb.y.domain.min = 50
        axb.y.domain.max = 1e4
    else:
        axb.y.domain.min = 10
        axb.y.domain.max = 10000
    
    # style axes
    for ax in (axb.x, axt.x, axb.y):
        ax.domain.show = False
        ax.ticks.show = True
        ax.ticks.near = 7.5
        ax.ticks.far = 0
        ax.ticks.labels.offset = 12
        ax.ticks.labels.style["font-size"] = 16
        ax.label.offset = 35
        ax.label.style["font-size"] = 16
        ax.ticks.style["stroke-width"] = 3
        ax.spine.style["stroke-width"] = 3

    avgdist = np.min([i.dist for i in sptree if not i.is_root()])
    axb.x.ticks.locator = toyplot.locator.Explicit(
        np.linspace(NEFF_MIN, NEFF_MAX, 4),
    )
    axt.x.ticks.locator = toyplot.locator.Explicit(
        np.linspace(NEFF_MIN, NEFF_MAX, 4),
        (avgdist / (2 * np.linspace(NEFF_MIN, NEFF_MAX, 4))).round(1),
    )
    axt.y.ticks.locator = toyplot.locator.Explicit([100, 1000, 10000])

    axb.hlines([100, 1000, 10000], style={"stroke": "grey", "stroke-width": 2, "stroke-dasharray": "6,6", "stroke-opacity": 0.7})

    # plot data
    axb.fill(
       data.index,
       [i[0] for i in data[smckey + "dist_CI95"]],
       [i[1] for i in data[smckey + "dist_CI95"]],
       opacity=0.33,
    )
    axb.plot(data.index, data[smckey + "dist"], stroke_width=3, color=color)
    style = dict(opacity=0.8, color='black', mstyle={"stroke": "none"})
    marks = [
        axb.scatterplot(data.index, data[smckey + "dist"], size=12, marker="o", color=color, opacity=0.7),
        axb.scatterplot(data.index, data[simkey + "dist"], size=6, marker='s', **style),
    ]
    return canvas

In [48]:
for nspecies in [1, 2, 8]:
    for smc in [True, False]:
        for topo in [True, False]:
            data = load_data(f"./validation_full_{nspecies}pops.npy")
            canvas = plot_distances(data, topo=topo, log=True, smc=smc)
            ttt = 'topo' if topo else 'tree'
            #toyplot.svg.render(canvas, f"../manuscript/figures/alternatives/validation-dists-{nspecies}pops-{ttt}.svg")


<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t53a17499a92c47efada8650f83f08554"> <polygon points="87.0,130.9312400111314 109.0,153.1211890276889 131.0,165.90656930834396 153.0,175.01155916025917 175.0,182.11778798530318 197.0,188.5434000261394 219.0,192.98242127727983 241.0,196.9346741929826 263.0,200.90619559064405 285.0,204.21472979109384 285.0,150.86776748619724 263.0,147.98732819799466 241.0,144.06714426077997 219.0,140.22340087140742 197.0,134.98843089719196 175.0,128.65953792973346 153.0,121.02457705626742 131.0,113.10783550852958 109.0,99.96387508172043 87.0,77.4497199329749" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:0.33;stroke:none"> 50000 200000 350000 500000 N e (diploid) 100 1000 10000 Distance to topo-change 5.0 1.2 0.7 0.5 Sptree edge lengths (coal units)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t93c80e92967b45e5925fd1ad8a8a5e3c"> <polygon points="87.0,152.66254941923904 109.0,174.8902772615859 131.0,187.59941426596254 153.0,196.4760425545723 175.0,204.4658370253567 197.0,209.64942689970553 219.0,215.1222856218903 241.0,218.95731240042852 263.0,222.97752114111876 285.0,226.23510769297377 285.0,176.75398425401104 263.0,173.22901483655752 241.0,170.07597152638445 219.0,165.52805562879533 197.0,160.83017259732873 175.0,154.3044294955837 153.0,147.3100809107918 131.0,138.09532902420082 109.0,124.5553027573649 87.0,102.6179329047641" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:0.33;stroke:none"> 50000 200000 350000 500000 N e (diploid) 100 1000 10000 Distance to tree-change 5.0 1.2 0.7 0.5 Sptree edge lengths (coal units)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t2efcb6cd2c364662b42c526cb1ca7a61"> <polygon points="87.0,130.9312400111314 109.0,153.1211890276889 131.0,165.90656930834396 153.0,175.01155916025917 175.0,182.11778798530318 197.0,188.5434000261394 219.0,192.98242127727983 241.0,196.9346741929826 263.0,200.90619559064405 285.0,204.21472979109384 285.0,150.86776748619724 263.0,147.98732819799466 241.0,144.06714426077997 219.0,140.22340087140742 197.0,134.98843089719196 175.0,128.65953792973346 153.0,121.02457705626742 131.0,113.10783550852958 109.0,99.96387508172043 87.0,77.4497199329749" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:0.33;stroke:none"> 50000 200000 350000 500000 N e (diploid) 100 1000 10000 Distance to topo-change 5.0 1.2 0.7 0.5 Sptree edge lengths (coal units)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t97cb4e63176b452fb8f70ae01acf960c"> <polygon points="87.0,152.66254941923904 109.0,174.8902772615859 131.0,187.59941426596254 153.0,196.4760425545723 175.0,204.4658370253567 197.0,209.64942689970553 219.0,215.1222856218903 241.0,218.95731240042852 263.0,222.97752114111876 285.0,226.23510769297377 285.0,176.75398425401104 263.0,173.22901483655752 241.0,170.07597152638445 219.0,165.52805562879533 197.0,160.83017259732873 175.0,154.3044294955837 153.0,147.3100809107918 131.0,138.09532902420082 109.0,124.5553027573649 87.0,102.6179329047641" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:0.33;stroke:none"> 50000 200000 350000 500000 N e (diploid) 100 1000 10000 Distance to tree-change 5.0 1.2 0.7 0.5 Sptree edge lengths (coal units)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t82593974c08d48dd80bcd8e37a45de89"> <polygon points="87.0,134.94464121772705 109.0,167.3375028021182 131.0,179.06418023276655 153.0,188.0368152069686 175.0,193.9627707597719 197.0,199.14586006828887 219.0,203.57787355452803 241.0,207.42206718596535 263.0,209.84623029060748 285.0,212.99600288597787 285.0,172.18638793917208 263.0,169.29522474182193 241.0,164.92546132414395 219.0,160.21685778345062 197.0,152.40222573059424 175.0,144.50979766852822 153.0,135.30920873939039 131.0,120.45402568527516 109.0,99.24506755968991 87.0,66.19412166091561" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:0.33;stroke:none"> 50000 200000 350000 500000 N e (diploid) 100 1000 10000 Distance to topo-change 5.0 1.2 0.7 0.5 Sptree edge lengths (coal units)

...

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t5733f55eac944139ac6a50aa417b0e7c"> <polygon points="87.0,168.2423641935958 109.0,187.58179571059867 131.0,198.73007577949568 153.0,206.97439762414044 175.0,212.32812880117845 197.0,217.48633574481687 219.0,221.90534259842687 241.0,226.022952204166 263.0,228.80311375742923 285.0,232.3228491931102 285.0,194.67392278462518 263.0,192.32733553664428 241.0,189.5127298303556 219.0,186.40961538736371 197.0,183.01729789513573 175.0,178.41571163332347 153.0,174.2493386323196 131.0,167.06785259931573 109.0,156.38791672331445 87.0,135.97626531480125" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:0.33;stroke:none"> 50000 200000 350000 500000 N e (diploid) 100 1000 10000 Distance to tree-change 5.0 1.2 0.7 0.5 Sptree edge lengths (coal units)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t65ef93805c084e6aa3baefe3cac12a29"> <polygon points="87.0,156.3477982963669 109.0,182.60787808989554 131.0,193.73645876643482 153.0,200.8421867937689 175.0,206.39578158544677 197.0,210.32646444776938 219.0,213.644428496024 241.0,216.3005311865045 263.0,219.19998034608915 285.0,221.7441912277899 285.0,196.53428460220786 263.0,194.2505668013531 241.0,191.45797145470607 219.0,188.13068425634066 197.0,183.80065180630538 175.0,178.27860692147394 153.0,170.40097674240926 131.0,156.19772809331627 109.0,133.0867957970476 87.0,75.80938202018669" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:0.33;stroke:none"> 50000 200000 350000 500000 N e (diploid) 100 1000 10000 Distance to topo-change 5.0 1.2 0.7 0.5 Sptree edge lengths (coal units)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tda503f57e1114a96ad5882103aaca686"> <polygon points="87.0,185.23076998240208 109.0,201.1592170891338 131.0,210.1809007436805 153.0,216.94499990168035 175.0,221.78957253535242 197.0,225.62060540472453 219.0,229.35007365684942 241.0,232.3769879755382 263.0,235.00429801005026 285.0,235.07942741708544 285.0,212.3139180620835 263.0,210.53967929810344 241.0,208.7300779811655 219.0,206.67676488491145 197.0,204.60256680809263 175.0,201.81493919710138 153.0,198.3905745975864 131.0,193.9684168192156 109.0,187.1932350256918 87.0,172.08138398648202" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:0.33;stroke:none"> 50000 200000 350000 500000 N e (diploid) 100 1000 10000 Distance to tree-change 5.0 1.2 0.7 0.5 Sptree edge lengths (coal units)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tc133337ca2a14bf0bc557e2ebc0fc164"> <polygon points="87.0,156.3477982963669 109.0,182.60787808989554 131.0,193.73645876643482 153.0,200.8421867937689 175.0,206.39578158544677 197.0,210.32646444776938 219.0,213.644428496024 241.0,216.3005311865045 263.0,219.19998034608915 285.0,221.7441912277899 285.0,196.53428460220786 263.0,194.2505668013531 241.0,191.45797145470607 219.0,188.13068425634066 197.0,183.80065180630538 175.0,178.27860692147394 153.0,170.40097674240926 131.0,156.19772809331627 109.0,133.0867957970476 87.0,75.80938202018669" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:0.33;stroke:none"> 50000 200000 350000 500000 N e (diploid) 100 1000 10000 Distance to topo-change 5.0 1.2 0.7 0.5 Sptree edge lengths (coal units)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t7cf44ee1fdab46d6a5afc58365d2ba54"> <polygon points="87.0,185.23076998240208 109.0,201.1592170891338 131.0,210.1809007436805 153.0,216.94499990168035 175.0,221.78957253535242 197.0,225.62060540472453 219.0,229.35007365684942 241.0,232.3769879755382 263.0,235.00429801005026 285.0,235.07942741708544 285.0,212.3139180620835 263.0,210.53967929810344 241.0,208.7300779811655 219.0,206.67676488491145 197.0,204.60256680809263 175.0,201.81493919710138 153.0,198.3905745975864 131.0,193.9684168192156 109.0,187.1932350256918 87.0,172.08138398648202" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:0.33;stroke:none"> 50000 200000 350000 500000 N e (diploid) 100 1000 10000 Distance to tree-change 5.0 1.2 0.7 0.5 Sptree edge lengths (coal units)

## Plot error

In [310]:
def plot_error(data_full, data_smc, topo:bool=False):
    """Plot the percent error for waiting distances between the analytical expectation
    and result of stochastic simulation, when data were simulated under either the
    full coalescent with recombination, or the SMC'.    
    """
    
    smckey = "topo_" if topo else "tree_"
    simkey = "topo_" if topo else "tree_"
    color = toytree.color.COLORS2[0]
    canvas = toyplot.Canvas(width=350, height=300)
    
    # setup axes
    axb = canvas.cartesian(margin=65)
    axt = axb.share("y")#, yscale="log")
    axb.x.label.text = "N<sub>e</sub> (diploid)"
    axt.x.label.text = "Sptree edge lengths (coal units)"
    axb.y.label.text = "Mean percent error"
    axb.x.domain.min = 0
    axb.y.domain.min = 0
    axb.y.domain.max = 10
    
    # style axes
    for ax in (axb.x, axt.x, axb.y):
        ax.domain.show = False
        ax.ticks.show = True
        ax.ticks.near = 7.5
        ax.ticks.far = 0
        ax.ticks.labels.offset = 12
        ax.ticks.labels.style["font-size"] = 16
        ax.label.offset = 35
        ax.label.style["font-size"] = 16
        ax.ticks.style["stroke-width"] = 3
        ax.spine.style["stroke-width"] = 3

    avgdist = np.min([i.dist for i in sptree if not i.is_root()])
    axb.x.ticks.locator = toyplot.locator.Explicit(
        np.linspace(NEFF_MIN, NEFF_MAX, 4),
    )
    axt.x.ticks.locator = toyplot.locator.Explicit(
        np.linspace(NEFF_MIN, NEFF_MAX, 4),
        (avgdist / (2 * np.linspace(NEFF_MIN, NEFF_MAX, 4))).round(1),
    )

    axb.hlines([5], style={"stroke": "grey", "stroke-width": 2, "stroke-dasharray": "6,6", "stroke-opacity": 0.7})

    # plot data
    axb.fill(
       data.index,
       data_full[smckey + "dist_error_perc"] - data_full[smckey + "dist_error_perc_sem"],
       data_full[smckey + "dist_error_perc"] + data_full[smckey + "dist_error_perc_sem"],
       opacity=0.33,
    )
    axb.fill(
       data.index,
       data_smc[smckey + "dist_error_perc"] - data_smc[smckey + "dist_error_perc_sem"],
       data_smc[smckey + "dist_error_perc"] + data_smc[smckey + "dist_error_perc_sem"],
       opacity=0.33, color='black',
    )
    style = dict(opacity=0.8, color='black', mstyle={"stroke": "none"})
    marks = [
        axb.plot(data_full.index, data_full[smckey + "dist_error_perc"], size=6, marker="o", color=color, opacity=0.7),
        axb.plot(data_smc.index, data_smc[simkey + "dist_error_perc"], size=6, marker='s', **style),
    ]
    return canvas

In [313]:
for nspecies in [1, 2, 8]:
    for topo in [True, False]:
        fdata = load_data(f"./validation_full_{nspecies}pops.npy")
        sdata = load_data(f"./validation_smc_{nspecies}pops.npy")
        canvas = plot_error(fdata, sdata, topo=topo)
        #toyplot.svg.render(canvas, "../manuscript/figures/alternatives/error-{nspecies}-{topo}.svg")

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t8a270838c9b04db993baac8b9641d3a8"> <polygon points="87.0,108.45497983669333 109.0,135.8589023050559 131.0,135.50532605831577 153.0,152.02631149473592 175.0,150.10113041110694 197.0,145.10051859255088 219.0,154.2774502658205 241.0,154.6870912350011 263.0,159.14295243130323 285.0,148.07750632812383 285.0,140.27785901533613 263.0,153.0273780269839 241.0,147.44852871673106 219.0,146.56761968669733 197.0,135.5625271255762 175.0,140.8588088099689 153.0,142.57069677925085 131.0,121.72588391882033 109.0,121.69670411570894 87.0,81.75756026244031" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:0.33;stroke:none"> <polygon points="87.0,130.22260056507514 109.0,138.50438830133575 131.0,132.57026147644515 153.0,141.12217792087964 175.0,147.36554020909338 197.0,142.99474599136425 219.0,147.94411700918445 241.0,154.0564277774837 263.0,152.5659459383368 285.0,156.92043671855058 285.0,150.23117044682562 263.0,144.86749543800684 241.0,146.0901607334235 219.0,139.79696929953244 197.0,133.09404038313812 175.0,137.66718553580796 153.0,129.5403634422051 131.0,118.16166772161736 109.0,124.87154371186801 87.0,112.3706361354206" style="fill:rgb(0%,0%,0%);fill-opacity:1.0;opacity:0.33;stroke:none"> 50000 200000 350000 500000 N e (diploid) 0 5 10 Mean percent error 5.0 1.2 0.7 0.5 Sptree edge lengths (coal units)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t0f5946d29eef40429dabfe3ba0511e48"> <polygon points="87.0,121.17803171246788 109.0,144.5877966828463 131.0,152.32415018436984 153.0,158.5454887081958 175.0,154.1171960482352 197.0,149.77365890005652 219.0,161.31579652893168 241.0,161.35612231967633 263.0,166.98618474614852 285.0,161.49078006639076 285.0,156.29099071042734 263.0,162.5095118548285 241.0,155.91379777620003 219.0,155.30434388891945 197.0,142.59217294738417 175.0,146.7032155486443 153.0,151.97082176158432 131.0,143.81466364053 109.0,134.10299824149132 87.0,101.32248911677176" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:0.33;stroke:none"> <polygon points="87.0,135.39637491202927 109.0,143.39242711334308 131.0,151.51575956638413 153.0,149.36013482306188 175.0,159.22203624235829 197.0,158.3361251583684 219.0,160.70300577772588 241.0,162.08784491817073 263.0,165.6088691041961 285.0,167.25786643201806 285.0,162.5747514141657 263.0,160.80587264316324 241.0,156.53733586785492 219.0,155.20809813323407 197.0,151.34354078762536 175.0,152.44128382207256 153.0,141.18259831412718 131.0,142.92486102873806 109.0,131.9640739935215 87.0,121.09479035936627" style="fill:rgb(0%,0%,0%);fill-opacity:1.0;opacity:0.33;stroke:none"> 50000 200000 350000 500000 N e (diploid) 0 5 10 Mean percent error 5.0 1.2 0.7 0.5 Sptree edge lengths (coal units)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t673a7d57a80744f3882139201b0071e6"> <polygon points="87.0,119.59749324422332 109.0,118.58623910153278 131.0,153.97953051778222 153.0,145.57764048274697 175.0,149.16456608515784 197.0,148.9624025034597 219.0,154.17844768664568 241.0,160.4148228568383 263.0,159.7381412955129 285.0,166.3554518526389 285.0,161.1908331787157 263.0,153.93352949768072 241.0,154.47312495206364 219.0,146.8797075834671 197.0,140.24259181546512 175.0,137.9556382847491 153.0,136.1443232239988 131.0,145.6833663502705 109.0,100.09092451009816 87.0,98.67710971092775" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:0.33;stroke:none"> <polygon points="87.0,118.91821691722285 109.0,136.89032688728605 131.0,143.2853719366662 153.0,144.0933102272186 175.0,153.3062430682106 197.0,153.33807106686024 219.0,156.98612942061837 241.0,155.27767521796127 263.0,159.0823237443225 285.0,159.5978156416505 285.0,153.86763755368284 263.0,152.8038560717507 241.0,148.79270156307098 219.0,150.66740298342694 197.0,145.42408366639648 175.0,145.9100694411489 153.0,134.57733520041722 131.0,130.82191074835362 109.0,122.34244178718876 87.0,85.90296666442718" style="fill:rgb(0%,0%,0%);fill-opacity:1.0;opacity:0.33;stroke:none"> 50000 200000 350000 500000 N e (diploid) 0 5 10 Mean percent error 5.0 1.2 0.7 0.5 Sptree edge lengths (coal units)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t3b83e63003774e1b939872ae65f8344b"> <polygon points="87.0,140.62526861791105 109.0,145.10272249469907 131.0,160.75370590614077 153.0,160.21278384039698 175.0,160.10688653549076 197.0,164.08487290648063 219.0,165.1097503963599 241.0,168.00520852725572 263.0,167.91117507394742 285.0,172.76547235443468 285.0,168.8731022524409 263.0,163.66695781531902 241.0,163.6350633710206 219.0,160.123327014976 197.0,158.87275054802282 175.0,154.34705316028015 153.0,154.1831679221839 131.0,154.6362423835842 109.0,136.4143346205846 87.0,130.35114713966638" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:0.33;stroke:none"> <polygon points="87.0,136.02452575117172 109.0,156.62600353781346 131.0,161.89918421814218 153.0,160.30549383546924 175.0,160.74651005499595 197.0,166.805849900899 219.0,163.9516862733915 241.0,169.12779883489029 263.0,169.98424389749016 285.0,166.36057630360168 285.0,162.0400039656756 263.0,165.6669625160025 241.0,164.81071567056625 219.0,159.07371365479736 197.0,161.80559182017515 175.0,155.19598656184297 153.0,154.5823734436058 131.0,155.64977547984196 109.0,149.49525862292052 87.0,123.81463437645255" style="fill:rgb(0%,0%,0%);fill-opacity:1.0;opacity:0.33;stroke:none"> 50000 200000 350000 500000 N e (diploid) 0 5 10 Mean percent error 5.0 1.2 0.7 0.5 Sptree edge lengths (coal units)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t3ed4d3a124d04c658ac01b51050c6626"> <polygon points="87.0,140.43023710831346 109.0,143.9916194771479 131.0,148.47231507914447 153.0,157.58213336003305 175.0,156.07485746810025 197.0,164.45827039295403 219.0,166.51761091674922 241.0,164.18167359311946 263.0,168.46502237865084 285.0,168.1166251259855 285.0,163.64441531991878 263.0,164.11658286837815 241.0,159.42736188823878 219.0,161.78139990170052 197.0,159.4980570742057 175.0,150.07145360749504 153.0,151.30233290806342 131.0,140.30040260984035 109.0,133.60679519928496 87.0,119.66358678869732" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:0.33;stroke:none"> <polygon points="87.0,112.59795730013033 109.0,145.04818168652795 131.0,157.22445523316168 153.0,158.0151669357711 175.0,160.05209858868477 197.0,164.94856327420678 219.0,164.77127507592803 241.0,164.22262264002507 263.0,165.3826795192956 285.0,169.11066333175881 285.0,164.89851752567762 263.0,160.68082041913198 241.0,159.2893301660214 219.0,159.90981771812878 197.0,159.8911028032184 175.0,154.48753687705926 153.0,152.02124750425477 131.0,149.85545587582288 109.0,134.7845807592299 87.0,75.79314842887747" style="fill:rgb(0%,0%,0%);fill-opacity:1.0;opacity:0.33;stroke:none"> 50000 200000 350000 500000 N e (diploid) 0 5 10 Mean percent error 5.0 1.2 0.7 0.5 Sptree edge lengths (coal units)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="te0200ec647e74ad6849a18f14fd3d1ed"> <polygon points="87.0,155.42153904824124 109.0,157.96008417200602 131.0,162.2234342231768 153.0,167.63564525554017 175.0,165.61480943253395 197.0,173.0633565169452 219.0,174.3999913411174 241.0,173.2467948218022 263.0,176.17237019634626 285.0,175.8869532615762 285.0,172.5609345003336 263.0,172.92609158160343 241.0,169.66609983467904 219.0,170.85284475302436 197.0,169.30098096927642 175.0,161.22924366121612 153.0,163.15610432952846 131.0,157.22203745674858 109.0,152.0296949332879 87.0,148.29973686456896" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:0.33;stroke:none"> <polygon points="87.0,153.19240725046768 109.0,162.56830192307876 131.0,170.03360452847562 153.0,167.93685196877934 175.0,168.72618894545494 197.0,171.49262233068308 219.0,172.87601173802926 241.0,172.9485194028206 263.0,172.60027230116123 285.0,175.36461272062593 285.0,172.10837385421985 263.0,169.09564777797874 241.0,169.30959875480028 219.0,169.2320901633941 197.0,167.64898561664302 175.0,164.49795660688466 153.0,163.63424369355621 131.0,165.65246805292009 109.0,157.22057461061758 87.0,145.86177160523258" style="fill:rgb(0%,0%,0%);fill-opacity:1.0;opacity:0.33;stroke:none"> 50000 200000 350000 500000 N e (diploid) 0 5 10 Mean percent error 5.0 1.2 0.7 0.5 Sptree edge lengths (coal units)